### Import Libraries

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
if len(tf.config.list_physical_devices('GPU')) > 0:
    print("GPU is Available!" )
else:
    raise Exception("No GPU available") 

GPU is Available!


### Hyperparameters

In [3]:
BATCH_SIZE = 32
DROPOUT_RATE = 0.5
ACTIVATION_FUNCTION = 'softmax'
LEARNING_RATE = 1e-4
LOSS_FUNCTION = 'categorical_crossentropy'

### Data Processing and Augmentation

Training data: `127, 433` images</br>
Test data: `27,307` images</br>
Validation data: `27,307` images

In [4]:
# Define image size and batch size
IMG_HEIGHT, IMG_WIDTH = 224, 224  # ResNet-50 input size

# Define data augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# No augmentation for validation and test sets, only rescaling
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_generator = train_datagen.flow_from_directory(
    '../../dataset/train',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    '../../dataset/validation',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    '../../dataset/test',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 127433 images belonging to 27 classes.
Found 19860 images belonging to 27 classes.
Found 49500 images belonging to 27 classes.


### Load Pretrained ResNet-50 Model

Potential to experiment with other pretrained ResNet models from Keras. OPtions currently offerred are:

| Model | Size (MB) | Top-1 Accuracy | Top-5 Accuracy | Parameters | Depth | Time (ms) per inference step (CPU) | Time (ms) per inference step (GPU) |
| --- | --- | --- | --- | --- | --- | --- | --- |
| ResNet50    |	98  | 74.9% | 92.1% | 25.6M | 107 |	58.2 |	4.6 |
| ResNet50V2  |	98  | 76.0% | 93.0% | 25.6M | 103 |	45.6 |	4.4 |
| ResNet101   |	171 | 76.4% | 92.8% | 44.7M | 209 |	89.6 |	5.2 |
| ResNet101V2 |	171 | 77.2% | 93.8% | 44.7M | 205 |	72.7 |	5.4 |
| ResNet152   |	232 | 76.6%	| 93.1%	| 60.4M	| 311 | 127.4|	6.5 |

Deeper models are recommended when one has access to more computational resources and larger datasets.

In [8]:
# Load the ResNet-50 model without the top classification layer
base_model = keras.applications.ResNet50(
    weights='imagenet',  # Load weights pre-trained on ImageNet
    include_top=False,   # Do not include the ImageNet classifier at the top
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)


### Customize Model for Classification

In [7]:
# Freeze the base model
base_model.trainable = False

# Create a new model on top
inputs = keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(DROPOUT_RATE)(x)  # Regularization
outputs = layers.Dense(train_generator.num_classes, activation=ACTIVATION_FUNCTION)(x)
model = keras.Model(inputs, outputs)


NameError: name 'base_model' is not defined

### Compile Model

In [7]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=LOSS_FUNCTION,
    metrics=['accuracy']
)


### Train the Model

In [10]:
# Define callbacks
callbacks = [
    keras.callbacks.ModelCheckpoint(
        'best_model.h5', save_best_only=True, monitor='val_accuracy', mode='max'
    ),
    keras.callbacks.EarlyStopping(
        monitor='val_accuracy', patience=5, restore_best_weights=True
    )
]

In [8]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=callbacks
)


Epoch 1/10
3983/3983 [==============================] - 1481s 371ms/step - loss: 3.3625 - accuracy: 0.0512 - val_loss: 3.1671 - val_accuracy: 0.0970
Epoch 2/10
3983/3983 [==============================] - 1165s 293ms/step - loss: 3.2126 - accuracy: 0.0708 - val_loss: 3.1229 - val_accuracy: 0.1023
Epoch 3/10
3983/3983 [==============================] - 1160s 291ms/step - loss: 3.1770 - accuracy: 0.0821 - val_loss: 3.1000 - val_accuracy: 0.1086
Epoch 4/10
3983/3983 [==============================] - 1161s 292ms/step - loss: 3.1538 - accuracy: 0.0870 - val_loss: 3.0804 - val_accuracy: 0.1195
Epoch 5/10
3983/3983 [==============================] - 1167s 293ms/step - loss: 3.1385 - accuracy: 0.0932 - val_loss: 3.0604 - val_accuracy: 0.1281
Epoch 6/10
3983/3983 [==============================] - 1160s 291ms/step - loss: 3.1237 - accuracy: 0.0989 - val_loss: 3.0542 - val_accuracy: 0.1437
Epoch 7/10
3983/3983 [==============================] - 1167s 293ms/step - loss: 3.1133 - accuracy: 0.1022

### Resume Training

In [5]:
model = keras.models.load_model('best_model.h5')

### Unfreeze Some Layers for Fine-Tuning

In [12]:
# Unfreeze the top layers of the model
base_model.trainable = True

# Freeze all layers except the last few layers
for layer in base_model.layers[:-10]:
    layer.trainable = False

# Recompile the model with a lower learning rate
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Continue training
history_fine = model.fit(
    train_generator,
    initial_epoch=6,  # Start from the epoch you left off
    epochs=10,
    validation_data=validation_generator,
    callbacks=callbacks
)


Epoch 7/10
3983/3983 [==============================] - 1200s 301ms/step - loss: 2.6295 - accuracy: 0.2193 - val_loss: 2.6915 - val_accuracy: 0.2127
Epoch 8/10
3983/3983 [==============================] - 1153s 289ms/step - loss: 2.6117 - accuracy: 0.2232 - val_loss: 2.5853 - val_accuracy: 0.2291
Epoch 9/10
3983/3983 [==============================] - 1144s 287ms/step - loss: 2.5937 - accuracy: 0.2283 - val_loss: 2.5953 - val_accuracy: 0.2327
Epoch 10/10
3983/3983 [==============================] - 1149s 288ms/step - loss: 2.5767 - accuracy: 0.2324 - val_loss: 2.5445 - val_accuracy: 0.2424


### Evaluate the Model

In [13]:
# Load the best model
model = keras.models.load_model('best_model.h5')

# Evaluate on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc:.2f}')

# Generate classification report
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
print('Classification Report')
print(classification_report(test_generator.classes, y_pred, target_names=test_generator.class_indices.keys()))


1547/1547 [==============================] - 342s 220ms/step - loss: 2.5122 - accuracy: 0.2508
Test accuracy: 0.25
1547/1547 [==============================] - 173s 111ms/step
Classification Report
                         precision    recall  f1-score   support

        dry-asphalt-bad       0.27      0.24      0.25      2350
       dry-asphalt-good       0.24      0.19      0.22      2350
   dry-asphalt-horrible       0.14      0.13      0.14       800
       dry-concrete-bad       0.23      0.32      0.26      2350
      dry-concrete-good       0.31      0.10      0.15      2350
  dry-concrete-horrible       0.33      0.45      0.38      2350
             dry-gravel       0.31      0.32      0.32      2350
                dry-mud       0.27      0.12      0.17      2350
             fresh_snow       0.36      0.77      0.49      2350
                    ice       0.30      0.19      0.23      2350
            melted_snow       0.60      0.39      0.47      2350
      water-asphalt-b

c:\Users\kyles\miniconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\kyles\miniconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\kyles\miniconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### VIsualize Training Results

In [14]:
# Plot training & validation accuracy values
plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'] + history_fine.history['accuracy'])
plt.plot(history.history['val_accuracy'] + history_fine.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.figure(figsize=(8, 6))
plt.plot(history.history['loss'] + history_fine.history['loss'])
plt.plot(history.history['val_loss'] + history_fine.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


NameError: name 'history' is not defined

<Figure size 800x600 with 0 Axes>

Failed because training and fine tuning was interrupted. Therefore, visualiztions of history could not be shown.

### Save Trained Model

In [15]:
# Save the trained model for future use
model.save('road_surface_classifier.h5')
